<div style="
    border: 3px solid white;
    border-radius: 16px;
    padding: 10px;
    background-color: #4F2683;
    color: white;
    max-width: 1300px;
    margin: auto;
">

<p align="center" style="margin: 0;">
  <img src="WesternEng.png" 
       alt="Western University Logo" width="200">
</p>

<h2 align="center" style="color:white; margin: 6px 0 2px 0;">Western University</h2>
<h3 align="center" style="color:white; margin: 2px 0;">Faculty of Engineering</h3>
<h4 align="center" style="color:white; margin: 2px 0;">Department of Electrical & Computer Engineering</h4>

<hr style="border: 1px solid white; margin: 8px 0;">

<div align="center" style="margin: 0; padding: 0; line-height: 1.3;">

**AISE-3350 — Cyber-Physical Systems Theory**  
Instructor: **Elvis Chen**  
Date: *[Insert Date]*  
Group Members: *Jonathan Das,*  

</div>

<hr style="border: 1px solid white; margin: 8px 0;">

</div>



## Table Of Contents
***
##### [1. Introduction](#introduction)
##### [2. Methods](#methods)
-   [Strategic Framework](#strategic-framework-and-loss-minimization-logic)
-   [Strategy-Valuation Adjustment](#strategy-valuation-adjustment)
-   [Hand Detection and Gesture Classification](#hand-detection-and-gesture-classification)
-   [Player Segmentation Logic](#player-segmentation-logic)
-   [System Integration](#system-integration)
##### [3. Machine Learning Model](#machine-learning-model)
-   [Dataset Pre-Processing](#dataset-pre-processing)
-   [Model Training](#model-training)
-   [Model Performance on Static Dataset](#model-performance-on-static-dataset)
##### [4. System Integration - Live Usage and Testing](#live-usage)
##### [5. Broader Impacts & Ethics](#broader-impacts-and-ethical-considerations)



<br><br><br>



## Introduction
---

Cyber-physical systems (CPS) consist of three entities, sensing, computing, and actuating working together to enable real-time processing and response. This project integrates computer vision and corresponding game theory to develop a CPS suited to play Rock-Paper-Scissors-Minus-One (RPS-1).

The more commonly known variant Rock-Paper-Scissors (RPS) offers a simple pairwise competition between two players. The “Minus-One” (RPS-1) variant introduces complexity to this decision space by introducing one additional hand per player. Players must present both of their hands and then remove one, with the remaining pair being the decider. This addition makes the game a two-stage, simultaneous, zero-sum game.  

The game of RPS-1 was chosen as it requires the project to apply all pillars of CPS. Sensing using Computer Vision (CV) and Machine Learning (ML) to detect the player’s actions. Computation to identify gestures and compute the game theory principles and payoff matrices. Lastly, a decision recommending what the player should do to maximize their chances of winning.

## **Methods**
---
## Overview

The objective of the CPS is (1) to identify the hand shapes being presented by each player for stage one and (2) to determine the optimal action to win the game through a game-theoretical analysis. The workflow consists of three major components: **hand detection and classification**, **enumeration of all possible outcomes**, and **strategy selection**.

The system first detects up to four hands simultaneously and classifies each as *Rock*, *Paper*, or *Scissors*. From these detected shapes, all feasible hand-combinations are computed using Cartesian enumeration. A payoff analysis is then applied to determine which move Player 1 should remove in Stage Two to minimize expected loss and maximize strategic advantage. This modular structure isolates perception from decision-making, allowing both components to be improved independently.


---
## **Strategic Framework and Loss Minimization Logic**

#### **Decision and Outcome Representation**


To formally model the restricted RPS-1 game, a payoff function is defined following standard RPS rules. For any pair of moves \(p_1\) (Player 1) and \(p_2\) (Player 2), the utility assigned to Player 1 is:

$$
U(p_1,p_2)=
\begin{cases}
+1 & \text{if } p_1 \text{ beats } p_2, \\
0 & \text{if } p_1 = p_2, \\
-1 & \text{if } p_2 \text{ beats } p_1.
\end{cases}
\tag{1}
$$

The system computes the Cartesian product of available actions:

$$
\Omega = P_1 \times P_2
\tag{2}
$$

where \(P_1\) and \(P_2\) are the move subsets available to each player. This generates all possible Stage Two interactions and their payoffs.

#### **Outcome Enumeration and Expected Statistics**

For each move \(m \in P_1\), the system counts the number of wins, losses, and draws:

$$
W_m = |\{(m,p_2) \in \Omega : U(m,p_2)=+1\}|, \tag{3}
$$

$$
L_m = |\{(m,p_2) \in \Omega : U(m,p_2)=-1\}|, \tag{4}
$$

$$
D_m = |\{(m,p_2) \in \Omega : U(m,p_2)=0\}|. \tag{5}
$$

From this, a basic expected value (risk-neutral utility) is computed:

$$
EV(m) = \frac{W_m - L_m}{|\Omega|}.
\tag{6}
$$

Equation (6) quantifies the relative strength or weakness of each move in the restricted subgame.

#### **Loss Minimization Logic**

If all outcomes in \( \Omega \) satisfy:

$$
U(p_1,p_2) \ge 0 \quad \forall (p_1,p_2)\in\Omega,
\tag{7}
$$

then no losing scenario exists, and the system concludes:

**All roads lead to victory**.

If losses exist, the system isolates the losing moves:

$$
L = \{(p_1,p_2)\in\Omega : U(p_1,p_2) = -1\}.
\tag{8}
$$

Each losing move is grouped by the Player 1 action that created it:

$$
\mathcal{L}(m) = |\{(m,p_2)\in L\}|.
\tag{9}
$$

If one move \(m\) accounts for all losses:

$$
\mathcal{L}(m) = |L|,
\tag{10}
$$

the system immediately recommends dropping that move.

---

## **Strategy-Valuation Adjustment**

To incorporate strategic preferences, the system defines a **strategy-dependent utility transformation**, modifying the payoff of each outcome according to Player 2's most likely reply.

#### **Opponent Move Likelihood Model**

The likelihood score for each opponent move is computed from:

$$
v(p_2) = |\{(p_1,p_2)\in L\}| - |\{(p_1,p_2)\in W\}|,
\tag{11}
$$

which emphasizes actions by Player 2 that historically cause Player 1 to lose.

The predicted reply is:

$$
p_2^* = \arg\max_{p_2\in P_2} v(p_2).
\tag{12}
$$

#### **Strategy Profile Weighting**

Each strategy assigns six weights:

$$
[w_s, \; l_s, \; d_s, \; w_o, \; l_o, \; d_o],
\tag{13}
$$

where:

- \(w_s, l_s, d_s\) apply when player 2 plays the **predicted move** \(p_2^*\),
- \(w_o, l_o, d_o\) apply when player 2 chooses **other moves**.

The transformed utility for each outcome \((p_1,p_2,U)\) becomes:

If \(p_2 = p_2^*\):

$$
U'(p_1,p_2)=
\begin{cases}
w_s & U=+1, \\
-l_s & U=-1, \\
d_s & U=0,
\end{cases}
\tag{14}
$$

Else:

$$
U'(p_1,p_2)=
\begin{cases}
w_o & U=+1, \\
-l_o & U=-1, \\
d_o & U=0.
\end{cases}
\tag{15}
$$

#### **Weighted Move Evaluation**

For each Player 1 move:

$$
\text{WeightedValue}(m)
 = \sum_{\substack{(p_1,p_2)\in\Omega\\p_1=m}} U'(p_1,p_2).
\tag{16}
$$

The **recommended move to drop** is the one with minimal value:

$$
m_{\text{drop}} = \arg\min_{m\in P_1} \text{WeightedValue}(m).
\tag{17}
$$

This ensures Player 1 removes the move most detrimental under the selected strategy profile.

---

## **Hand Detection and Gesture Classification**

Three gesture-recognition methods were evaluated:

1. **Mediapipe Landmark Classifier** – fast, rule-based, no training required - Ultimatley rejected due to its non-generalizability .
2. **Mediapipe Gesture Recognition Model** – rejected due to poor robustness to lighting and gesture ambiguity.
3. **Custom YOLOv11 Gesture Detector** – trained specifically on RPS gestures for real-time accuracy.

YOLOv11 was chosen due to high-speed multi-object detection and domain-specific fine-tuning through:

- Image annotation via custom scripts,
- Automatic label generation and dataset structuring,
- Controlled training of a YOLOv11 backbone for 80 epochs.

The resulting model outputs symbolic labels suitable for the decision-making engine.

---

## **Player Segmentation Logic**

YOLO detection uses **spatial partitioning**:

- Upper frame regions → Player 2
- Lower frame regions → Player 1

This allows clean assignment of gestures even with simultaneous detections, assuming a fixed camera orientation.

---

## **System Integration**

After gesture detection, moves are passed to the decision engine, where all combinations of Stage Two interactions are evaluated using equations (1)–(17). The computed recommendation is then overlayed on the live video feed using OpenCV, enabling real-time feedback.

Modular separation of:

- Detection,
- Classification,
- Segmentation,
- Strategy evaluation,

ensures ease of debugging, reproducibility, and adherence to CPS design principles.

---


## **Machine Learning Model**
---

## **Dataset Pre-Processing**
[View Pre-Processing Step 1](modelTraining\imageannotation.py)\
[View Pre-Processing Step 2](modelTraining\datasplit.py)


In our implementation, we selected **YOLOv11** because it proved to be the most robust model for identifying and labeling up to four hands in a single frame. To make YOLOv11 suitable for our RPS-1 application, we trained it on numerous hand-gesture images consisting of the three target classes: **Rock**, **Paper**, and **Scissors**. Due to the similarity in publicly available datasets on Kaggle, three datasets were merged to reduce the chance of overfitting and increase generalizability.

To account for variations in image formats, `imageannotation.py` automatically handles JPG, JPEG, and PNG files. A pretrained YOLOv11 model is used to localize the hand in each image and draw a bounding box around it. These bounding boxes provide clear supervision signals during training, ensuring the model focuses on the gesture-performing region.

Following annotation, `datasplit.py` divides the dataset into an **80% training** and **20% validation** split, randomly shuffling samples before placement into new directories. Because YOLOv11 requires a YAML configuration file, `datasplit.py` generates this file automatically for training via `rps_model.py`.

---
## **Model Training**  
 [View Training Code](modelTraining\rps_model.py)


The training pipeline uses a pretrained YOLOv11s model as its foundation. Starting from a pretrained backbone significantly accelerates convergence because the model already understands general visual patterns such as edges, shapes, and textures. This allows the network to focus on learning the distinctions between the three gesture classes, even with a limited dataset.

A dataset YAML file defines the training and validation directories as well as the class labels. During the training process, YOLOv11 automatically handles dataloading, batch creation, and optimization, while continuously monitoring performance metrics such as Loss, Precision, Recall, and mean Average Precision (mAP).

### **Data Augmentation Strategy**

To ensure that the model generalizes well to real-world camera conditions, a comprehensive augmentation pipeline is applied during training. These augmentations include:

- **HSV colour variation** to simulate different lighting environments  
- **Rotations, translations, scaling, and shearing** to vary the spatial relationships of the hands  
- **Perspective distortion** to represent off-angle camera viewpoints  
- **Horizontal and vertical flips** to account for mirrored gestures  
- **Mosaic augmentation** to expose the model to multiple contexts simultaneously  
- **MixUp blending** to regularize decision boundaries  

This augmentation strategy minimizes overfitting and enhances robustness against noise, motion, and inconsistent hand positioning. Furthermore, the use of mosic augmentation specifically allows the model to learn from images containing more than one hand which is important to consider given the task at hand and the limitations of the dataset.

### **Hyperparameter Configuration**

Training is performed for 80 epochs at a resolution of 640×640 with a batch size of 16. A cosine learning-rate scheduler is used to smooth the convergence behaviour over time, and early stopping is enabled to prevent unnecessary computation if validation performance plateaus. These hyperparameters offer a practical balance between training speed and detection accuracy.

### **Model Export and Integration**

Upon completion of training, YOLOv11 automatically exports the best checkpoint—based on validation results—into a dedicated project folder. This exported weight file becomes the inference model used in the real-time hand-detection system of the CPS. By integrating this optimized model into the perception pipeline, the system achieves:

- Fast and accurate gesture detection  
- Reliable multi-hand tracking  
- Robust performance in general indoor lighting  
- Smooth interaction with the downstream decision-making engine  

### **Significance to the CPS Project**

The YOLOv11 training pipeline is an essential component of the RPS-1 system. It transforms raw video input into symbolic gesture predictions, enabling the game decision engine to operate on high-confidence inputs. The pipeline demonstrates how modern computer-vision models can be tailored effectively to domain-specific tasks, even with mixed datasets and limited hardware resources.

Overall, this training process provides the foundation for a responsive, accurate, and highly adaptable gesture-recognition subsystem within the broader CPS architecture.

---


## **Model Performance on Static Dataset**

### **F1-Confidence Curve** *(Figure 1)*

<img src="rps_yolo11/F1_curve.png" width="450">

Figure 1 illustrates the F1-confidence curve for the YOLOv11 gesture classifier. The model stabilizes above an **F1 score of 0.95** between confidence intervals of **0.2 and 0.8**, showing excellent robustness. The optimal performance appears at a confidence threshold of **approximately 0.4**, where the model reaches an F1 score of **~0.98**.

The **rock** gesture consistently shows the lowest F1 score, though still high enough to avoid significant gameplay disruption.

---

### **Precision-Recall Curve** *(Figure 2)*

<img src="rps_yolo11/PR_curve.png" width="450">

Figure 2 presents the Precision–Recall curve demonstrating the model's reliability across the full recall spectrum. The model achieves **mAP = 0.984 at IoU = 0.5**, highlighting excellent predictive performance. Paper and scissors gestures achieve AP values of **0.993** and **0.991**, respectively. Although rock underperforms relative to these two, its AP of **0.967** demonstrates strong detectability.

---

## **Confusion Matrices** *(Figures 3 and 4)*

### **Figure 3 — Raw Confusion Matrix**

<img src="rps_yolo11/confusion_matrix.png" width="450">

### **Figure 4 — Normalized Confusion Matrix**

<img src="rps_yolo11/confusion_matrix_normalized.png" width="450">

Figures 3 and 4 further validate classification performance. The raw confusion matrix reveals that the **rock** gesture experiences the highest misclassification count (~40 errors). Conversely, **paper** and **scissors** demonstrate far fewer errors (~10–15).

Normalized results indicate:

- **Paper** accuracy: **1.00**
- **Scissors** accuracy: **0.99**
- **Rock** accuracy: **0.97**

These high values confirm the consistency and reliability of the YOLOv11 detector.




# **Live Usage**
[View Computer Vision Script](RPSCamera.py)\
[View Game Logic](RPSLogic.py)



## **Broader Impacts and Ethical Considerations**

Although this project may initially appear as a playful gesture-based game, its underlying technologies—computer vision, gesture recognition, real-time decision systems, and human–machine interaction—have broad implications extending well beyond the Rock–Paper–Scissors context.

Gesture-recognition interfaces can transform how users interact with machines by enabling **hands-free**, **intuitive**, and **accessible** control mechanisms. This has applications in:

- Operating rooms (sterile, hands-free environments)  
- Assistive technologies for individuals with limited mobility  
- Robotics and industrial automation  
- Retail systems (touchless kiosks)  
- Educational tools leveraging physical interaction  

By relying on **low-cost webcam hardware** and **open-source ML frameworks**, this project demonstrates that advanced computer vision systems can be developed without expensive infrastructure, lowering adoption barriers for small companies and research groups.

### **Economic and Ethical Considerations**

Widespread adoption of AI-driven, gesture-based systems introduces economic and ethical challenges:

- **Workforce displacement** due to automation  
- The need for **upskilling** to support AI systems  
- **Privacy concerns** regarding image capture  
- **Bias risks** if training datasets are not diverse  
- The need for transparent, explainable AI in public-use interfaces  

Addressing these challenges is essential for responsible implementation and sustaining public trust in vision-based AI systems.
